# Keras学习
## keras 函数式API

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

函数式编程与keras.Sequential 相比而言，更加灵活
深度学习模型通常是一个DAG的层次模型，函数式API是其构建层次图的工具

In [2]:
# minst数据
# 输入数据 是784维的一维向量 ,批次大小通常需要忽略，因为需要根据实际的数据集大小来划分
inputs = keras.Input(shape=(784,))
# 也可以输入为一个32*32*3的tensor张量
# img_inputs = keras.Input(shape=(32,32,3))
inputs.dtype

tf.float32

In [3]:
inputs.shape

TensorShape([None, 784])

In [4]:
# 创建隐含层
dense = layers.Dense(64,activation='relu')
x = dense(inputs)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(10)(x)

In [5]:
model = keras.Model(inputs=inputs,outputs=outputs,name='mnist_model')
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 开始训练
# 
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000,784).astype("float32")/255
x_test = x_test.reshape(10000,784).astype("float32")/255



In [7]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

In [8]:
history=model.fit(x_train,y_train,batch_size=64,epochs=2,validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 2s 40us/sample - loss: 0.3629 - accuracy: 0.8960 - val_loss: 0.1924 - val_accuracy: 0.9423
Epoch 2/2
48000/48000 [==============================] - 1s 30us/sample - loss: 0.1592 - accuracy: 0.9538 - val_loss: 0.1541 - val_accuracy: 0.9557


In [9]:
test_scores = model.evaluate(x_test,y_test,verbose=2)
print('Test loss:%s' % test_scores[0])
print('Test accuracy:%s' % test_scores[1])

10000/1 - 0s - loss: 0.0831 - accuracy: 0.9546
Test loss:0.15269442020617424
Test accuracy:0.9546


In [10]:
# 保存模型和加载模型
model.save('mnist_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mnist_model\assets


In [11]:
del model
model = keras.models.load_model('mnist_model')
test_scores = model.evaluate(x_test,y_test,verbose=2)
print('Test loss:%s' % test_scores[0])
print('Test accuracy:%s' % test_scores[1])

10000/1 - 0s - loss: 0.0831 - accuracy: 0.9546
Test loss:0.15269442020617424
Test accuracy:0.9546


## 使用相同的图定义多个模型

In [12]:
encoder_input = keras.Input(shape=(28,28,1),name='img')
# Conv2D(filters,kernel_size) filters:是指的featuremap的层数，kernel_size指的是卷积核的大小比如 3*3，默认步长为（1，1），padding方式默认为valid
x=layers.Conv2D(16,3,activation='relu')(encoder_input)
x=layers.Conv2D(32,3,activation='relu')(x)
# 最大池化层 MaxPooling2D(pool_size,strides):pool_size 可以是tuple 或者是integer，代表池化层的大小，默认strides为None，即和pool_size相同，padding方式默认为valid
x=layers.MaxPooling2D(3)(x)
x=layers.Conv2D(32,3,activation='relu')(x)
x=layers.Conv2D(16,3,activation='relu')(x)
# GlobalMaxPooling2D() 全局池化，池化层的大小为整个feature-map的大小，最终得到一个1*1*feature-map个数的张量
encoder_output=layers.GlobalMaxPooling2D()(x)
encoder = keras.Model(encoder_input,encoder_output,name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0   

In [13]:
x=layers.Reshape((4,4,1))(encoder_output)
# 反卷积 将数据扩充，卷积的输入输出，在这里可看作反卷积的 输出 输入
x=layers.Conv2DTranspose(16,3,activation='relu')(x)
x=layers.Conv2DTranspose(32,3,activation='relu')(x)
# 上采样
x=layers.UpSampling2D(3)(x)
x=layers.Conv2DTranspose(16,3,activation='relu')(x)
decoder_output=layers.Conv2DTranspose(1,3,activation='relu')(x)
autodecoder = keras.Model(encoder_input,decoder_output,name='autodecoder')
autodecoder.summary()

Model: "autodecoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                

定义好模型后 ，你可以像使用layer一样使用模型

## 构建复杂的图拓扑
### 定义有多个输入和输出的模型

In [14]:
num_tags=12
num_words=10000
num_departments=4
title_input=keras.Input(shape=(None,),name='title')
body_input=keras.Input(shape=(None,),name='body')
tags_input=keras.Input(shape=(num_tags,),name='tags')
# embedding
title_features=layers.Embedding(num_words,64)(title_input)
body_features=layers.Embedding(num_words,64)(body_input)
title_features=layers.LSTM(128)(title_features)
body_features=layers.LSTM(32)(body_features)
x=layers.concatenate([title_features,body_features,tags_input])
priority_pred=layers.Dense(1,name='priority')(x)
department_pred=layers.Dense(num_departments,name='department')(x)
model=keras.Model(inputs=[title_input,body_input,tags_input],outputs=[priority_pred,department_pred],name='multimodel')
model.summary()

Model: "multimodel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
body (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      title[0][0]                      
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      body[0][0]                       
_________________________________________________________________________________________

In [15]:
keras.utils.plot_model(model,'multi_input_and_output_model.png',show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [16]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "priority": keras.losses.BinaryCrossentropy(from_logits=True),
        "department": keras.losses.CategoricalCrossentropy(from_logits=True),
    },
    loss_weights=[1.0, 0.2],
)

In [17]:
# Dummy input data
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype("float32")

# Dummy target data
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

model.fit(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_targets, "department": dept_targets},
    epochs=2,
    batch_size=32,
)

Train on 1280 samples
Epoch 1/2
1280/1280 [==============================] - 5s 4ms/sample - loss: 1.3280 - priority_loss: 0.7011 - department_loss: 3.1343
Epoch 2/2
1280/1280 [==============================] - 2s 2ms/sample - loss: 1.3087 - priority_loss: 0.6968 - department_loss: 3.0595


## 构建一个简单的ResNet网络

In [18]:
inputs = keras.Input(shape=(32,32,3),name='img')
# 第一块block
x=layers.Conv2D(32,3,activation='relu')(inputs)
x=layers.Conv2D(64,3,activation='relu')(x)
block_1_output=layers.MaxPooling2D(3)(x)
# 第二块block
x=layers.Conv2D(64,3,activation='relu',padding='same')(block_1_output)
x=layers.Conv2D(64,3,activation='relu',padding='same')(x)
block_2_output=layers.add([x,block_1_output])
# 第三块block
x=layers.Conv2D(64,3,activation='relu',padding='same')(block_2_output)
x=layers.Conv2D(64,3,activation='relu',padding='same')(x)
block_3_output=layers.add([x,block_2_output])
# 输出
x=layers.Conv2D(64,3,activation='relu')(block_3_output)
x=layers.GlobalAveragePooling2D()(x)
x=layers.Dense(256,activation='relu')(x)
x=layers.Dropout(0.5)(x)
outputs=layers.Dense(10)(x)

model=keras.Model(inputs,outputs,name='cifar10_resnet')
model.summary()


Model: "cifar10_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 30, 30, 32)   896         img[0][0]                        
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 64)   18496       conv2d_4[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 9, 9, 64)     0           conv2d_5[0][0]                   
_____________________________________________________________________________________

In [19]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- unknown url type: https